In [1]:
import csv
import os
import shutil
import pandas as pd
import numpy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

Reading data from given files

In [2]:

# Define data path
Data_Path = 'fuzzy_csv'
Fuzzy_data_files = ['Clubs_data.csv','Organisers_In_Fests.csv','Participants_In_Fests.csv']
Original_Metadata = pd.read_csv('Metadata.csv')
#fclub_df = pd.read_csv('Clubs_data.csv')
Dictionary_Fuzzified_PD={}
Dictionary_Defuzzified_PD={}
for i in Fuzzy_data_files:
    File_path = Data_Path+'/'+i
    Iterating_FDF = pd.read_csv(File_path)
    Dictionary_Fuzzified_PD[i] = Iterating_FDF
    Dictionary_Defuzzified_PD[i] = Iterating_FDF
    
#len(fclub_df['Name'])

Function to Defuzzify given Data

In [3]:
def Defuzzify(Fuzzy_dataframe,Metadata):
    Length = len(Fuzzy_dataframe['Name'])
    for i in range(Length):
        #print(Fuzzy_dataframe['Name'][i],end=' # ')
        Metaname= process.extractOne(Fuzzy_dataframe['Name'][i], Metadata['Name'])[0]
        Fuzzy_dataframe['Name'][i] = Metaname
        #print(Fuzzy_dataframe['Name'][i])
    return Fuzzy_dataframe



In [ ]:
for i in Dictionary_Fuzzified_PD:
    Dictionary_Defuzzified_PD[i] = Defuzzify(Dictionary_Fuzzified_PD[i].copy(),Original_Metadata)

Saving Defuzzified CSV files

In [5]:
Defuzzy_Path = 'defuzzy_csv'
shutil.rmtree(Defuzzy_Path,ignore_errors=True)   
os.makedirs(Defuzzy_Path) 
for i in Dictionary_Defuzzified_PD:
    Dictionary_Defuzzified_PD[i].to_csv(Defuzzy_Path+'/'+i, index=False)

Sorting the dataset

In [6]:
import pandas as pd
#Sorting Data
def Sorting_CSV(SourceCSV):
    sortread = pd.read_csv(Defuzzy_Path+'/'+SourceCSV)
    print(Defuzzy_Path+'/'+SourceCSV)
    sortread.sort_values(["Name"],axis=0,ascending=[True],inplace=True)
    sortread.to_csv(Defuzzy_Path+'/'+SourceCSV,index=False)


Sorting_CSV("Clubs_data.csv")
Sorting_CSV("Organisers_In_Fests.csv")
Sorting_CSV("Participants_In_Fests.csv")


defuzzy_csv/Clubs_data.csv
defuzzy_csv/Organisers_In_Fests.csv
defuzzy_csv/Participants_In_Fests.csv


Finding Number of Distinct Clubs and Fests

In [7]:
DF = pd.read_csv(Defuzzy_Path+'/'+"Clubs_data.csv")
Distinct_Clubs_Count = len(DF['Club_Name'].unique())
DF = pd.read_csv(Defuzzy_Path+'/'+"Organisers_In_Fests.csv")
Distinct_Fests_Count = len(DF['Fest_Name'].unique())

print(Distinct_Clubs_Count)
print(Distinct_Fests_Count)



3
2


Finding number of distinct club events 

In [8]:
DF = pd.read_csv(Defuzzy_Path+'/'+"Clubs_data.csv")
Distinct_ClubEvent_Count= []
for i in range(1,Distinct_Clubs_Count+1):
    print(i)
    DF_Iterator = DF[DF['Event'].str.contains('club_'+str(i)+'_', na=False)]
    value = len(DF_Iterator['Event'].unique())
    Distinct_ClubEvent_Count.append(value)

print(Distinct_ClubEvent_Count)


1
2
3
[3, 3, 3]


Finding number of distinct Fest events

In [9]:
DF = pd.read_csv(Defuzzy_Path+'/'+"Participants_In_Fests.csv")
Distinct_FestEvent_Count= []
for i in range(1,Distinct_Fests_Count+1):
    print(i)
    DF_Iterator = DF[DF['Event'].str.contains('fest_'+str(i)+'_', na=False)]
    value = len(DF_Iterator['Event'].unique())
    Distinct_FestEvent_Count.append(value)

print(Distinct_FestEvent_Count)

1
2
[13, 15]


In [10]:
rows = [[Distinct_Clubs_Count],[Distinct_Fests_Count],Distinct_ClubEvent_Count,Distinct_FestEvent_Count]
with open("COUNT_VAR.csv", 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(rows)

Function for creating a dictionary for json conversion

In [12]:
def df_append(Dictionary_Defuzzified_PD,Original_Metadata,i):
    #dflen = len(Original_Metadata)
    dfname = Original_Metadata['Name'][i]
    dfroll = Original_Metadata['ID'][i]
    club_org=[]
    club_par=[]
    fest_org=[]
    fest_par=[]
    #For Clubs
    for j in range(len(Dictionary_Defuzzified_PD['Clubs_data.csv'])):
        if ((Dictionary_Defuzzified_PD['Clubs_data.csv'].iloc[j]['Name'] == dfname) and(Dictionary_Defuzzified_PD['Clubs_data.csv'].iloc[j]['Role'].__contains__('org'))):
            club_org.append(Dictionary_Defuzzified_PD['Clubs_data.csv'].iloc[j]['Club_Name'])
        if ((Dictionary_Defuzzified_PD['Clubs_data.csv'].iloc[j]['Name'] == dfname) and(Dictionary_Defuzzified_PD['Clubs_data.csv'].iloc[j]['Role'].__contains__('Par'))):
            club_par.append(Dictionary_Defuzzified_PD['Clubs_data.csv'].iloc[j]['Event'])
    #print(club_org)
    #print(club_par)
    
    lsss=[]
    dic_profile={}
    dic_club={}
    dic_profile['name']=dfname
    dic_profile['id']=dfroll

    for j in range(Distinct_Clubs_Count):
        dic_club_i={}
        club_name = 'club_'+str(j+1)
        if club_name in club_org:
            dic_club_i['isOrganiser']='Organiser'
        else:
            dic_club_i['isOrganiser']=''
        
        for k in range(Distinct_ClubEvent_Count[j]):
            dic_club_par={}
            club_event_name = club_name +'_event_'+str(k+1)
            if club_event_name in club_par:
                dic_club_par['participated']=True
            else:
                dic_club_par['participated']=False
            dic_club_i[club_event_name]=dic_club_par
        dic_club[club_name]=dic_club_i
        dic_profile['clubs']=dic_club
    
    #For Fests     
    for j in range(len(Dictionary_Defuzzified_PD['Organisers_In_Fests.csv'])):
        if ((Dictionary_Defuzzified_PD['Organisers_In_Fests.csv'].iloc[j]['Name'] == dfname) ):
            fest_org.append(Dictionary_Defuzzified_PD['Organisers_In_Fests.csv'].iloc[j]['Fest_Name'])
    for j in range(len(Dictionary_Defuzzified_PD['Participants_In_Fests.csv'])):
        if ((Dictionary_Defuzzified_PD['Participants_In_Fests.csv'].iloc[j]['Name'] == dfname) ):
            fest_par.append(Dictionary_Defuzzified_PD['Participants_In_Fests.csv'].iloc[j]['Event'])
    dic_fest={}
    for j in range(Distinct_Fests_Count):
        dic_fest_i={}
        fest_name = 'fest_'+str(j+1)
        if fest_name in fest_org:
            dic_fest_i['isOrganiser']='Organiser'
        else:
            dic_fest_i['isOrganiser']=''
        for k in range(Distinct_FestEvent_Count[j]):
            dic_fest_par={}
            fest_event_name = fest_name +'_event_'+str(k+1)
            if fest_event_name in fest_par:
                dic_fest_par['participated']=True
                
            else:
                dic_fest_par['participated']=False
            dic_fest_i[fest_event_name]=dic_fest_par
        dic_fest[fest_name]=dic_fest_i
        dic_profile['fests']=dic_fest      
    #print(dic_profile)
    
    
    return dic_profile
    

totalls=[]
for i in range(len(Original_Metadata)):
    totalls.append(df_append(Dictionary_Defuzzified_PD,Original_Metadata,i))
    
    

In [13]:
totalls

[{'name': 'Abagail Marx',
  'id': '18XJ1A0202',
  'clubs': {'club_1': {'isOrganiser': '',
    'club_1_event_1': {'participated': True},
    'club_1_event_2': {'participated': True},
    'club_1_event_3': {'participated': False}},
   'club_2': {'isOrganiser': '',
    'club_2_event_1': {'participated': True},
    'club_2_event_2': {'participated': True},
    'club_2_event_3': {'participated': False}},
   'club_3': {'isOrganiser': '',
    'club_3_event_1': {'participated': True},
    'club_3_event_2': {'participated': False},
    'club_3_event_3': {'participated': False}}},
  'fests': {'fest_1': {'isOrganiser': '',
    'fest_1_event_1': {'participated': True},
    'fest_1_event_2': {'participated': True},
    'fest_1_event_3': {'participated': True},
    'fest_1_event_4': {'participated': True},
    'fest_1_event_5': {'participated': True},
    'fest_1_event_6': {'participated': True},
    'fest_1_event_7': {'participated': True},
    'fest_1_event_8': {'participated': False},
    'fest_1

JSON

In [16]:
import json

In [17]:
with open('Profilejson.json', 'w') as f:
    # indent does the prettify
    json.dump(totalls, f, indent=4)

In [18]:

organiser_club = pd.read_csv("defuzzy_csv/Clubs_data.csv")
organiser_club.drop(organiser_club[organiser_club['Role'] == "Participant"].index, inplace = True)
organiser_club.to_csv("Organiser_Final.csv",index=False)